# Part 3: Deployment & Monitoring (SageMaker)

## Overview
This notebook covers the **Deployment** and **Monitoring** phases of the MLOps lifecycle for the Olist E-Commerce project. 

**Objectives:**
1. **Deploy** the XGBoost model trained in `02_Modeling.ipynb` to a real-time SageMaker endpoint.
2. **Enable Data Capture** to log all inference requests and predictions to S3.
3. **Implement Model Monitoring**:
   - **Data Quality Monitor**: Detects drift in input features (e.g., changes in `payment_value_sum` distribution).
   - **Model Quality Monitor**: continually evaluates model performance (Accuracy, F1, AUC) by comparing predictions against ground truth labels.
4. **Visualize** the monitoring results and CloudWatch metrics.

> ** COST WARNING**: This notebook creates a real-time endpoint (`ml.m5.xlarge`) and monitoring schedules. These resources incur hourly costs. **Run the Cleanup section at the end of this notebook to delete these resources.**

**Reference**: This implementation is adapted from `lab-5-1-model-monitoring-with-sagemaker-and-cloudwatch`.


### Setup & Configuration
Import necessary libraries and configure S3 bucket locations. Prefixes must match those used in previous notebooks (`01` and `02`) to ensure correct retrieval of data and model artifacts.


In [6]:
# Setup
import sagemaker
import boto3
import os
import time
import json
import pandas as pd
import numpy as np
import awswrangler as wr
from datetime import datetime, timedelta
from time import sleep
from threading import Thread

from sagemaker import image_uris, get_execution_role
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sagemaker.model_monitor import (
    DataCaptureConfig,
    DefaultModelMonitor,
    ModelQualityMonitor,
    DatasetFormat,
    EndpointInput,
    CronExpressionGenerator,
)
from sagemaker.s3 import S3Downloader, S3Uploader

sm_sess = sagemaker.Session()
region = sm_sess.boto_region_name
try:
    role = get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

bucket = sm_sess.default_bucket()

# Prefixes (Must align with 02_Modeling)
prefix = "datalake/olist/monitoring"
model_prefix = "modeling/output"
data_capture_prefix = f"{prefix}/datacapture"
reports_prefix = f"{prefix}/reports"
base_prefix = f"s3://{bucket}/modeling/xgb-baseline/"

print(f"Region: {region}")
print(f"Role: {role}")
print(f"Bucket: {bucket}")

sched_name = "olist-data-quality-monitor"
model_sched_name = "olist-model-quality-monitor"

Region: us-east-1
Role: arn:aws:iam::587322031938:role/LabRole
Bucket: sagemaker-us-east-1-587322031938


#### Output Verification
Verify that the outputs display the expected AWS Region, IAM Execution Role ARN, and default S3 bucket name.


### 1. Load Trained Model Artifact
Locate the latest model trained in `02_Modeling.ipynb` to avoid retraining. The function `get_latest_model_artifact` scans the output directory and picks the most recent `model.tar.gz`.


In [7]:
s3_client = boto3.client('s3')

def get_latest_model_artifact(bucket, prefix):
    resp = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    contents = sorted(resp.get('Contents', []), key=lambda x: x['LastModified'], reverse=True)
    for c in contents:
        if c['Key'].endswith('/output/model.tar.gz'):
            return f"s3://{bucket}/{c['Key']}"
    return None

model_data = get_latest_model_artifact(bucket, model_prefix)

if not model_data:
    raise ValueError(f"No model artifact found in s3://{bucket}/{model_prefix}. Please run 02_Modeling.ipynb first.")

print(f"Using model artifact: {model_data}")

# Training image URI (XGBoost)
image_uri = image_uris.retrieve(framework="xgboost", region=region, version="1.5-1")

Using model artifact: s3://sagemaker-us-east-1-587322031938/modeling/output/sagemaker-xgboost-2026-02-17-19-39-09-126/output/model.tar.gz


#### Model Confirmation
The output above should display the S3 URI of the model artifact found (e.g., `s3://.../model.tar.gz`). If it fails, ensure that you have successfully run the training step in `02_Modeling.ipynb`.

### 2. Live Deployment with Data Capture
Deploy the model to an `ml.m5.xlarge` instance and enable **Data Capture** to save all inputs and outputs to S3 for monitoring.

**Safety Check**: If the endpoint already exists, we skip deployment to avoid errors and duplicate costs.


In [8]:
endpoint_name = "olist-xgb-monitoring-ep"
data_capture_uri = f"s3://{bucket}/{data_capture_prefix}"

sm_client = boto3.client("sagemaker")

try:
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    print(f"Endpoint {endpoint_name} already exists. Status: {resp['EndpointStatus']}")
    predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=sm_sess, serializer=CSVSerializer())
except sm_client.exceptions.ClientError:
    print(f"Creating new endpoint: {endpoint_name}...")
    
    model = Model(
        image_uri=image_uri,
        model_data=model_data,
        role=role,
        sagemaker_session=sm_sess
    )
    
    data_capture_config = DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=data_capture_uri
    )
    
    model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.xlarge",
        endpoint_name=endpoint_name,
        data_capture_config=data_capture_config
    )
    predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=sm_sess, serializer=CSVSerializer())

Endpoint olist-xgb-monitoring-ep already exists. Status: InService


#### Deployment Status
If the endpoint was already running, you will see `Endpoint ... already exists. Status: InService`. If it is new, SageMaker will print a series of dashes (`-`) indicating the provisioning progress. Once complete, the endpoint is ready for real-time inference.

In [9]:
#NEW CELL- Infrastructure Monitoring
# This cell creates CloudWatch alarms to monitor the health of the
# SageMaker endpoint, including invocation errors, inference latency,
# and CPU/memory utilization. These monitors detect infrastructure-
# level failures that can impact ML system reliability and availability.


import boto3, json
cw = boto3.client("cloudwatch", region_name=region)

# Common endpoint dimensions
# NOTE: VariantName is often "AllTraffic" unless you set a custom variant name.
endpoint_dims = [
    {"Name": "EndpointName", "Value": endpoint_name},
    {"Name": "VariantName", "Value": "AllTraffic"},
]

alarm_topic_arn = None  # Optional: set to an SNS topic ARN to notify you

def put_alarm(name, metric, stat="Average", period=60, eval_periods=5, threshold=1, comp="GreaterThanThreshold"):
    args = dict(
        AlarmName=name,
        MetricName=metric,
        Namespace="AWS/SageMaker",
        Dimensions=endpoint_dims,
        Statistic=stat,
        Period=period,
        EvaluationPeriods=eval_periods,
        Threshold=threshold,
        ComparisonOperator=comp,
        TreatMissingData="notBreaching",
    )
    if alarm_topic_arn:
        args["AlarmActions"] = [alarm_topic_arn]
        args["OKActions"] = [alarm_topic_arn]
    cw.put_metric_alarm(**args)

# 4XX/5XX errors (bad requests / server errors)
put_alarm(f"{endpoint_name}-4xx", "Invocation4XXErrors", stat="Sum", threshold=5)
put_alarm(f"{endpoint_name}-5xx", "Invocation5XXErrors", stat="Sum", threshold=1)

# Latency alarms (tune thresholds to your use case)
put_alarm(f"{endpoint_name}-model-latency", "ModelLatency", stat="Average", threshold=2000)      # ms
put_alarm(f"{endpoint_name}-overhead-latency", "OverheadLatency", stat="Average", threshold=500) # ms

# Capacity / saturation style signals (optional but common)
put_alarm(f"{endpoint_name}-cpu-high", "CPUUtilization", stat="Average", threshold=80)
put_alarm(f"{endpoint_name}-mem-high", "MemoryUtilization", stat="Average", threshold=80)

print("Created/updated infrastructure alarms for errors, latency, CPU, and memory.")


Created/updated infrastructure alarms for errors, latency, CPU, and memory.


In [10]:
# CloudWatch Monitoring Dashboard
# This cell creates a CloudWatch dashboard to visualize
# infrastructure, performance, and monitoring metrics for the
# deployed SageMaker endpoint, including traffic, errors,
# latency, resource utilization, and model monitoring signals.

dashboard_name = f"{endpoint_name}-mlops-dashboard"

dashboard_body = {
  "widgets": [
    {
      "type": "metric",
      "x": 0, "y": 0, "width": 12, "height": 6,
      "properties": {
        "metrics": [
          [ "AWS/SageMaker", "Invocations", "EndpointName", endpoint_name, "VariantName", "AllTraffic" ],
          [ ".", "Invocation4XXErrors", ".", ".", ".", "." ],
          [ ".", "Invocation5XXErrors", ".", ".", ".", "." ],
        ],
        "period": 60,
        "stat": "Sum",
        "region": region,
        "title": "Endpoint traffic & errors"
      }
    },
    {
      "type": "metric",
      "x": 12, "y": 0, "width": 12, "height": 6,
      "properties": {
        "metrics": [
          [ "AWS/SageMaker", "ModelLatency", "EndpointName", endpoint_name, "VariantName", "AllTraffic" ],
          [ ".", "OverheadLatency", ".", ".", ".", "." ],
        ],
        "period": 60,
        "stat": "Average",
        "region": region,
        "title": "Endpoint latency (ms)"
      }
    },
    {
      "type": "metric",
      "x": 0, "y": 6, "width": 12, "height": 6,
      "properties": {
        "metrics": [
          [ "AWS/SageMaker", "CPUUtilization", "EndpointName", endpoint_name, "VariantName", "AllTraffic" ],
          [ ".", "MemoryUtilization", ".", ".", ".", "." ],
        ],
        "period": 60,
        "stat": "Average",
        "region": region,
        "title": "Endpoint instance utilization"
      }
    },
    {
      "type": "metric",
      "x": 12, "y": 6, "width": 12, "height": 6,
      "properties": {
        "metrics": [
          # Model Monitor emits metrics when enable_cloudwatch_metrics=True.
          # These commonly appear under AWS/SageMaker and/or a ModelMonitor-related namespace depending on SDK/version.
          # If you don't see them, keep the dashboard but verify exact namespace/dimensions in CloudWatch Metrics UI.
          [ "AWS/SageMaker", "ConstraintViolations", "MonitoringSchedule", sched_name ],
          [ ".", "ConstraintViolations", ".", model_sched_name ],
        ],
        "period": 3600,
        "stat": "Sum",
        "region": region,
        "title": "Model Monitor violations (data/model schedules)"
      }
    }
  ]
}

cw.put_dashboard(DashboardName=dashboard_name, DashboardBody=json.dumps(dashboard_body))
print("Created CloudWatch dashboard:", dashboard_name)



Created CloudWatch dashboard: olist-xgb-monitoring-ep-mlops-dashboard


### 3. Data Quality Monitoring
Create a baseline using the training dataset (from `02_Modeling`) to detect if the distribution of incoming data shifts significantly (Data Drift).


In [11]:
# Load training data for baseline (Must match what was used in 02_Modeling)
train_uri = f"{base_prefix}train/"
baseline_results_uri = f"s3://{bucket}/{prefix}/baselining/data_quality"

print(f"Baselining with training data: {train_uri}")

data_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

# Check if baseline already exists to run expensive job only once
existing_baseline = s3_client.list_objects_v2(Bucket=bucket, Prefix=f"{prefix}/baselining/data_quality/statistics.json")
if existing_baseline.get('KeyCount', 0) > 0:
    print("Found existing Data Quality Baseline. Skipping baseline job.")
    # Attach to existing monitor logic would go here if we needed the object, 
    # but we can reuse the monitor object for scheduling.
else:
    print("Starting Data Quality Baseline Job...")
    data_monitor.suggest_baseline(
        baseline_dataset=train_uri,
        dataset_format=DatasetFormat.csv(header=False),
        output_s3_uri=baseline_results_uri,
        wait=True,
        logs=False
    )

Baselining with training data: s3://sagemaker-us-east-1-587322031938/modeling/xgb-baseline/train/


INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2026-02-17-20-07-21-544


Starting Data Quality Baseline Job...
...........................................................!

#### Baseline Job Status
The code checks for an existing `statistics.json` file. If found, it skips the baselining job to save time. Otherwise, it launches a Processing Job to compute statistics (mean, variance, etc.) and constraints (type check, null checks) on the training data.

#### Schedule Data Quality Monitor
Schedule the Data Quality Monitor to run hourly. It analyzes the data captured from the endpoint (`data_capture_uri`) and compares it against the baseline. If violations are found (e.g., drift), it generates a violation report.


In [12]:
# Schedule Data Quality Monitor
sched_name = "olist-data-quality-monitor"

try:
    sm_client.describe_monitoring_schedule(MonitoringScheduleName=sched_name)
    print(f"Schedule {sched_name} already exists.")
except sm_client.exceptions.ResourceNotFound:
    print(f"Creating Monitoring Schedule: {sched_name}")
    data_monitor.create_monitoring_schedule(
        monitor_schedule_name=sched_name,
        endpoint_input=EndpointInput(
            endpoint_name=endpoint_name,
            destination="/opt/ml/processing/input/endpoint_data",
        ),
        output_s3_uri=f"s3://{bucket}/{reports_prefix}/data_quality",
        schedule_cron_expression=CronExpressionGenerator.hourly(),
        enable_cloudwatch_metrics=True,
    )

Creating Monitoring Schedule: olist-data-quality-monitor


INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: olist-data-quality-monitor


#### Schedule Confirmation
Ensure the output confirms that `olist-data-quality-monitor` has been created or already exists.


### 4. Model Quality Monitoring
To monitor model performance (e.g., Accuracy), **Ground Truth** labels are required. In a real scenario, these arrive after inference (e.g., did the package actually arrive late?). Here, ground truth is simulated for demonstration.


In [15]:
model_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sm_sess
)

# Create baseline for Model Quality (requires predictions + labels)
# Skip the explicit `suggest_baseline` step here to save time/cost in this demo, 
# as it requires merging validation predictions with labels manually first. 
# Instead, we'll proceed to creating the schedule which assumes we upload ground truth later.
pass

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


#### Schedule Model Quality Monitor
Create a schedule to monitor model performance, specifying `BinaryClassification` as the problem type. The monitor matches inference IDs from the endpoint requests with ground truth data found in the `ground_truth_input` S3 path.


In [16]:
model_sched_name = "olist-model-quality-monitor"
ground_truth_path = f"s3://{bucket}/{prefix}/ground_truth_data"

try:
    sm_client.describe_monitoring_schedule(MonitoringScheduleName=model_sched_name)
    print(f"Schedule {model_sched_name} already exists.")
except sm_client.exceptions.ResourceNotFound:
    print(f"Creating Model Quality Schedule: {model_sched_name}")
    # Note: The same constraints from data quality are used for simplicity, 
    # or we could point to a pre-calculated constraints file. 
    # For this demo, we effectively monitor without strict baseline constraints just to show setup.
    
    model_monitor.create_monitoring_schedule(
        monitor_schedule_name=model_sched_name,
        endpoint_input=EndpointInput(
            endpoint_name=endpoint_name,
            inference_attribute="0",           # First column is probability
            probability_threshold_attribute=0.5,
            destination="/opt/ml/processing/input_data",
        ),
        problem_type="BinaryClassification",
        ground_truth_input=ground_truth_path,
        output_s3_uri=f"s3://{bucket}/{reports_prefix}/model_quality",
        schedule_cron_expression=CronExpressionGenerator.hourly(),
        enable_cloudwatch_metrics=True,
    )

Creating Model Quality Schedule: olist-model-quality-monitor


INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: olist-model-quality-monitor


#### Schedule Status
The output should confirm `olist-model-quality-monitor` creation.

### 5. Simulate Traffic & Ground Truth
Start a thread to send requests to the endpoint and upload corresponding simulated ground truth labels, providing data for the monitor to process. 

**Note:** `awswrangler` is used here to read the test dataset directly from S3, as `pd.read_csv` cannot handle S3 directory prefixes natively.


In [17]:
# Load some test data as a sample payload
# From 02_Modeling splits
test_uri = f"{base_prefix}test/"

# FIX: Use awswrangler to read from S3 prefix (native pandas fails on folders)
test_df = wr.s3.read_csv(test_uri, header=None)

sample_payloads = test_df.iloc[:10, 1:].to_csv(header=False, index=False).split("\n") # Drop label col 0

def simulate_traffic():
    print("Sending traffic... (Stop kernel to end)")
    for i, payload in enumerate(sample_payloads):
        if not payload: continue
        try:
            resp = predictor.predict(payload, inference_id=str(i)) # Inference ID is key for joining
            # print(f"Prediction {i}: {resp}")
            sleep(0.5)
        except Exception as e:
            print(f"Error: {e}")
            
# Run simulation once for demo
simulate_traffic()

INFO:awswrangler.distributed.ray._core:Initializing a Ray instance
2026-02-17 20:14:54,786	WARNING services.py:2070 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 1909395456 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.48gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2026-02-17 20:14:54,840	INFO worker.py:1852 -- Started a local Ray instance.


Sending traffic... (Stop kernel to end)


#### Traffic Simulation
The output will display "Sending traffic..." followed by successful execution. If errors occur, check the endpoint status in the SageMaker console.


## 6. Cleanup (Teardown)
**CRITICAL**: Run this cell to delete resources and avoid unexpected costs.

In [21]:
def cleanup():
    print("Cleaning up resources...")
    # Delete Schedules
    for schedule in [sched_name, model_sched_name]:
        try:
            sm_client.delete_monitoring_schedule(MonitoringScheduleName=schedule)
            print(f"Deleted schedule: {schedule}")
        except Exception as e:
            print(f"Skipped schedule {schedule}: {e}")
            
    # --- ADD THIS WAIT STEP ---
    print("Waiting for schedules to clear...")
    time.sleep(15) 
    # --------------------------
    
    # Delete Endpoint
    try:
        predictor.delete_endpoint(delete_endpoint_config=True) # Good practice to delete config too
        print(f"Deleted endpoint: {endpoint_name}")
    except Exception as e:
        print(f"Skipped endpoint: {e}")

cleanup()

Cleaning up resources...
Skipped schedule olist-data-quality-monitor: An error occurred (ResourceNotFound) when calling the DeleteMonitoringSchedule operation: Monitoring Schedule arn:aws:sagemaker:us-east-1:587322031938:monitoring-schedule/olist-data-quality-monitor not found
Skipped schedule olist-model-quality-monitor: An error occurred (ResourceNotFound) when calling the DeleteMonitoringSchedule operation: Monitoring Schedule arn:aws:sagemaker:us-east-1:587322031938:monitoring-schedule/olist-model-quality-monitor not found
Waiting for schedules to clear...


INFO:sagemaker:Deleting endpoint configuration with name: olist-xgb-monitoring-ep


Skipped endpoint: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "olist-xgb-monitoring-ep".


In [22]:
import boto3
sm = boto3.client("sagemaker")

# 1. Check Endpoint
try:
    sm.delete_endpoint(EndpointName=endpoint_name)
    print(f"Successfully deleted endpoint: {endpoint_name}")
except Exception as e:
    if "Could not find" in str(e) or "ResourceNotFound" in str(e):
        print("Endpoint is already gone.")
    else:
        print(f"Endpoint status: {e}")

# 2. Check Dashboard
try:
    cw = boto3.client("cloudwatch")
    cw.delete_dashboards(DashboardNames=[f"{endpoint_name}-mlops-dashboard"])
    print("Deleted CloudWatch Dashboard.")
except:
    pass

print("Cleanup complete.")

Successfully deleted endpoint: olist-xgb-monitoring-ep
Deleted CloudWatch Dashboard.
Cleanup complete.


I0000 00:00:1771360110.287907    4746 chttp2_transport.cc:1182] ipv4:169.255.255.2:37745: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {created_time:"2026-02-17T20:28:30.287891393+00:00", http2_error:2, grpc_status:14}
*** SIGTERM received at time=1771360112 on cpu 0 ***
PC: @     0x7f9e76e0de9e  (unknown)  epoll_wait
    @     0x7f9e1e66db0d         64  absl::lts_20240722::AbslFailureSignalHandler()
    @     0x7f9e76d2a520  (unknown)  (unknown)
[2026-02-17 20:28:32,664 E 4111 4111] logging.cc:497: *** SIGTERM received at time=1771360112 on cpu 0 ***
[2026-02-17 20:28:32,664 E 4111 4111] logging.cc:497: PC: @     0x7f9e76e0de9e  (unknown)  epoll_wait
[2026-02-17 20:28:32,665 E 4111 4111] logging.cc:497:     @     0x7f9e1e66db39         64  absl::lts_20240722::AbslFailureSignalHandler()
[2026-02-17 20:28:32,665 E 4111 4111] logging.cc:497:     @     0x7f9e76d2a520  (unknown)  (unknown)


#### Cleanup Confirmation
When you run the cleanup function, verify that it successfully deletes both schedules and the endpoint. This ensures no residual charges will accrue.